In [8]:
import os
import glob
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Audio
import librosa
import librosa.display

# Scikit learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix as sk_plot_confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import Input, add, Flatten, Dense, BatchNormalization, Dropout, LSTM, GRU
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalMaxPooling2D, Activation, LeakyReLU, ReLU, MaxPooling1D

# Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, BatchNormalization, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.utils import to_categorical

# TensorFlow
from tensorflow.keras.utils import to_categorical

# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

dataset = []
for folder in ["Heartbeat_Sounds/set_a/**","Heartbeat_Sounds/set_b/**"]:
    for filename in glob.iglob(folder):
        if os.path.exists(filename):
            label = os.path.basename(filename).split("_")[0]
            duration = librosa.get_duration(filename=filename)
            # skip audio smaller than 3 secs
            if duration>=3:
                slice_size = 3
                iterations = int((duration-slice_size)/(slice_size-1))
                iterations += 1
#                 initial_offset = (duration % slice_size)/2
                initial_offset = (duration - ((iterations*(slice_size-1))+1))/2
                if label not in ["Aunlabelledtest", "Bunlabelledtest"]:
                    for i in range(iterations):
                        offset = initial_offset + i*(slice_size-1)
                        
                        dataset.append({
                                "filename": filename,
                                "label": label,
                                "offset": offset
                            })
                       
                        
dataset = pd.DataFrame(dataset)
dataset = shuffle(dataset, random_state=42)
dataset.info()

# Function to extract features
def extract_features(audio_path, offset):
    y, sr = librosa.load(audio_path, offset=offset, duration=3)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)
    return mfccs

# Extract features for all data points
x_data = []
for idx in tqdm(range(len(dataset))):
    x_data.append(extract_features(dataset.filename.iloc[idx], dataset.offset.iloc[idx]))

# Convert to numpy array
x_data = np.asarray(x_data)

# Encode Labels
encoder = LabelEncoder()
encoder.fit(dataset.label)
y_data = encoder.transform(dataset.label)

unique_labels = dataset['label'].unique()
num_classes = len(unique_labels)

# Compute class weights
class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_data), y=y_data)

y_data_one_hot = to_categorical(y_data, num_classes=num_classes)


# Initialize StratifiedKFold with train_size and test_size
kfold = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Initialize lists to store results for each fold
train_accuracies = []
test_accuracies = []
train_auc_scores = []
test_auc_scores = []
train_mcc_scores = []
test_mcc_scores = []
train_conf_matrices = []
test_conf_matrices = []
train_class_reports = []
test_class_reports = []
cm_test_normalized = []
report_test_list = []
cm_test_normalized_list = []

# Open the common results file in 'w' mode to overwrite existing content
with open('results_heartSound/all_folds_lstm5_results.txt', 'w') as common_file:
    # Split the indices instead of the dataset
    for fold, (train_index, test_index) in enumerate(kfold.split(x_data, y_data_one_hot), 1):
        x_train_fold, x_test_fold = x_data[train_index], x_data[test_index]
        y_train_fold, y_test_fold = y_data_one_hot[train_index], y_data_one_hot[test_index]

        model = Sequential()

        model.add(Conv1D(2048, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(40, 130)))
        model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
        model.add(BatchNormalization())

        model.add(Conv1D(1024, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(40, 130)))
        model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
        lstm_model.add(BatchNormalization())

        model.add(Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
        model.add(BatchNormalization())

        model.add(LSTM(256, return_sequences=True))
        model.add(LSTM(128))


        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(num_classes, activation='softmax'))


        model.summary()

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        your_epochs = 300  # You can choose an appropriate number of epochs
        your_batch_size = 128  # You can choose an appropriate batch size

        # Train and evaluate the model for the current fold
        model.fit(x_train_fold, y_train_fold, epochs=your_epochs, batch_size=your_batch_size)
        y_pred_train = model.predict(x_train_fold)
        y_pred_test = model.predict(x_test_fold)
        
        y_pred_test_labels = np.argmax(y_pred_test, axis=1)
        report_test = classification_report(np.argmax(y_test_fold, axis=1), y_pred_test_labels, output_dict=True)

        # Calculate metrics for train set
        train_accuracy = accuracy_score(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_auc = roc_auc_score(y_train_fold, y_pred_train, multi_class='ovr')
        train_mcc = matthews_corrcoef(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_cm = confusion_matrix(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_class_report = classification_report(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1), target_names=encoder.classes_)

        # Calculate metrics for test set
        test_accuracy = accuracy_score(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_auc = roc_auc_score(y_test_fold, y_pred_test, multi_class='ovr')
        test_mcc = matthews_corrcoef(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_class_report = classification_report(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1), target_names=encoder.classes_)

        # Inside the loop where you append accuracies to lists
        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)

        # Calculate std for train and test accuracies
        train_accuracy_std = np.std(train_accuracies)
        test_accuracy_std = np.std(test_accuracies)


        # Save results to the common text file for both train and test sets
        common_file.write(f'Fold {fold} Results:\n')
        common_file.write(f'Train Accuracy: {train_accuracy} (std: {train_accuracy_std})\n')
        common_file.write(f'Train AUC Score: {train_auc}\n')
        common_file.write(f'Train MCC Score: {train_mcc}\n\n')
        common_file.write('Train Confusion Matrix:\n')
        common_file.write(str(train_cm))
        common_file.write('\n\nTrain Classification Report:\n')
        common_file.write(train_class_report)

        common_file.write(f'\n\nTest Accuracy: {test_accuracy} (std: {test_accuracy_std})\n')
        common_file.write(f'Test AUC Score: {test_auc}\n')
        common_file.write(f'Test MCC Score: {test_mcc}\n\n')
        common_file.write('Test Confusion Matrix:\n')
        common_file.write(str(test_cm))
        common_file.write('\n\nTest Classification Report:\n')
        common_file.write(test_class_report)
        
        # Print a separator between folds
        common_file.write("\n" + "="*40 + "\n")
        
        # Calculate average metrics for train set
        avg_train_accuracy = np.mean(train_accuracies)
        avg_train_auc = np.mean(train_auc_scores)
        avg_train_mcc = np.mean(train_mcc_scores)
        avg_train_conf_matrix = np.mean(train_conf_matrices, axis=0)  # Average confusion matrix

        # Convert continuous probabilities to class labels using argmax
        y_pred_train_class = np.argmax(y_pred_train, axis=1)

        # Calculate average metrics for test set
        avg_test_accuracy = np.mean(test_accuracies)
        avg_test_auc = np.mean(test_auc_scores)
        avg_test_mcc = np.mean(test_mcc_scores)
        avg_test_conf_matrix = np.mean(test_conf_matrices, axis=0)  # Average confusion matrix

        # Convert continuous probabilities to class labels using argmax
        y_pred_test_class = np.argmax(y_pred_test, axis=1)
        
        # Calculate average results for train set
        avg_train_class_report = classification_report(np.argmax(y_train_fold, axis=1), y_pred_train_class, target_names=encoder.classes_)

        # Calculate average results for test set
        avg_test_class_report = classification_report(np.argmax(y_test_fold, axis=1), y_pred_test_class, target_names=encoder.classes_)


        # Inside the loop where you plot confusion matrices
        plt.figure()
        train_cm = confusion_matrix(np.argmax(y_train_fold, axis=1), y_pred_train.argmax(axis=1))

        # Round each value in the confusion matrix to 5 decimal places
        rounded_train_cm = np.round(train_cm / np.sum(train_cm, axis=1)[:, np.newaxis], 5)

        sns.heatmap(rounded_train_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
        plt.title(f'Train Confusion Matrix - Fold {fold}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'LSTM/train_confusion_matrix_fold_{fold}.png')
        plt.close()

        # Inside the loop where you plot confusion matrices
        plt.figure()
        test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), y_pred_test.argmax(axis=1))

        # Round each value in the confusion matrix to 5 decimal places
        rounded_test_cm = np.round(test_cm / np.sum(test_cm, axis=1)[:, np.newaxis], 5)

        sns.heatmap(rounded_test_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
        plt.title(f'Test Confusion Matrix - Fold {fold}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'LSTM/test_confusion_matrix_fold_{fold}.png')
        plt.close()

        # Inside the loop where you append results to lists
        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)
        train_auc_scores.append(train_auc)
        test_auc_scores.append(test_auc)
        train_mcc_scores.append(train_mcc)
        test_mcc_scores.append(test_mcc)
        train_conf_matrices.append(train_cm)
        test_conf_matrices.append(test_cm)
        train_class_reports.append(train_class_report)
        test_class_reports.append(test_class_report)
        
        # Print a separator between folds
        common_file.write("\n" + "="*40 + "\n")
        
        # Append classification report and confusion matrix to lists
        report_test_list.append(report_test)
        cm_test_normalized_list.append(cm_test_normalized)
        

    # Calculate average metrics for train set
    avg_train_accuracy = np.mean(train_accuracies)
    avg_train_auc = np.mean(train_auc_scores)
    avg_train_mcc = np.mean(train_mcc_scores)

    # Reshape y_train_fold to (num_samples, num_classes)
    y_train_fold_reshaped = to_categorical(y_train_fold, num_classes=len(encoder.classes_))

    # Convert predictions to one-hot encoding
    y_pred_train_one_hot = to_categorical(np.argmax(y_pred_train, axis=1), num_classes=len(encoder.classes_))

    # Calculate average confusion matrix for train set
    avg_train_conf_matrix = np.mean(train_conf_matrices, axis=0).astype(int)


    # Calculate average classification report for train set
    avg_train_class_report = classification_report(
        np.argmax(y_train_fold, axis=1), 
        np.argmax(y_pred_train_one_hot, axis=1), 
        target_names=encoder.classes_, 
        digits=5
    )
    

    # Calculate average metrics for test set
    avg_test_accuracy = np.mean(test_accuracies)
    avg_test_auc = np.mean(test_auc_scores)
    avg_test_mcc = np.mean(test_mcc_scores)

    # Reshape y_test_fold to (num_samples, num_classes)
    y_test_fold_reshaped = to_categorical(y_test_fold, num_classes=len(encoder.classes_))

    # Convert predictions to one-hot encoding
    y_pred_test_one_hot = to_categorical(np.argmax(y_pred_test, axis=1), num_classes=len(encoder.classes_))

    # Calculate average confusion matrix for test set
    avg_test_conf_matrix = np.mean(test_conf_matrices, axis=0).astype(int)

    # Calculate average classification report for test set
    avg_test_class_report = classification_report(
        np.argmax(y_test_fold, axis=1), 
        np.argmax(y_pred_test_one_hot, axis=1), 
        target_names=encoder.classes_, 
        digits=5
    )

    # Inside the loop where you plot the average confusion matrix for the test set
    avg_cm_test_normalized = np.mean(cm_test_normalized_list, axis=0)
    plt.figure()

    avg_test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))

    # Normalize the confusion matrix by dividing each value by the sum of its row
    normalized_avg_test_cm = avg_test_cm / avg_test_cm.sum(axis=1)[:, np.newaxis]

    # Plot confusion matrix for the average test set
    plt.figure()
    sns.heatmap(normalized_avg_test_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
    plt.title(f'Average Test Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('LSTM/average_test_confusion_matrix.png')
    plt.close()



    # Save average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy:.5f} (std: {np.std(test_accuracies):.5f})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc:.5f}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc:.5f}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_cm_test_normalized.round(5).astype(int)))  # Display confusion matrix with 5 decimal places
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))


    # Print and write the average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy:.5f} (std: {np.std(test_accuracies):.5f})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc:.5f}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc:.5f}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_cm_test_normalized.round(5).astype(int)))  # Display confusion matrix with 5 decimal places
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))


    # Print and write the average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Train Accuracy: {avg_train_accuracy} (std: {np.std(train_accuracies)})\n')
    common_file.write(f'Average Train AUC Score: {avg_train_auc}\n')
    common_file.write(f'Average Train MCC Score: {avg_train_mcc}\n\n')
    common_file.write('Average Train Confusion Matrix:\n')
    common_file.write(str(avg_train_conf_matrix.astype(int)))
    common_file.write("\n\nAverage Train Classification Report:\n")
    common_file.write(str(avg_train_class_report))

    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy} (std: {np.std(test_accuracies)})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_test_conf_matrix.astype(int)))
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))
        
    # Print average results for train set
    print(f'Average Train Accuracy: {avg_train_accuracy}')
    print(f'Average Train AUC Score: {avg_train_auc}')
    print(f'Average Train MCC Score: {avg_train_mcc} \n')
    print("\nAverage Train Classification Report:")
    print(avg_train_class_report)

    # Print average results for test set
    print(f'Average Test Accuracy: {avg_test_accuracy}')
    print(f'Average Test AUC Score: {avg_test_auc}')
    print(f'Average Test MCC Score: {avg_test_mcc} \n')
    print("\nAverage Test Classification Report:")
    print(avg_test_class_report)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1441 entries, 168 to 1126
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  1441 non-null   object 
 1   label     1441 non-null   object 
 2   offset    1441 non-null   float64
dtypes: float64(1), object(2)
memory usage: 45.0+ KB


100%|███████████████████████████████████████████████████████████████████████████████| 1441/1441 [03:19<00:00,  7.22it/s]


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 40, 2048)          1333248   
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 20, 2048)         0         
 1D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 20, 2048)         8192      
 hNormalization)                                                 
                                                                 
 conv1d_12 (Conv1D)          (None, 20, 1024)          10486784  
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 10, 1024)         0         
 g1D)                                                            
                                                      

9/9 [==============================] - 123s 14s/step - loss: 0.0958 - accuracy: 0.9601
Epoch 137/300
9/9 [==============================] - 123s 14s/step - loss: 0.1064 - accuracy: 0.9661
Epoch 138/300
9/9 [==============================] - 125s 14s/step - loss: 0.1134 - accuracy: 0.9601
Epoch 139/300
9/9 [==============================] - 121s 14s/step - loss: 0.1050 - accuracy: 0.9661
Epoch 140/300
9/9 [==============================] - 121s 13s/step - loss: 0.0923 - accuracy: 0.9688
Epoch 141/300
9/9 [==============================] - 121s 13s/step - loss: 0.0896 - accuracy: 0.9566
Epoch 142/300
9/9 [==============================] - 122s 13s/step - loss: 0.0937 - accuracy: 0.9557
Epoch 143/300
9/9 [==============================] - 122s 14s/step - loss: 0.0595 - accuracy: 0.9757
Epoch 144/300
9/9 [==============================] - 122s 13s/step - loss: 0.0581 - accuracy: 0.9783
Epoch 145/300
9/9 [==============================] - 121s 14s/step - loss: 0.0794 - accuracy: 0.9670
Epoc

9/9 [==============================] - 126s 14s/step - loss: 0.0036 - accuracy: 0.9991
Epoch 299/300
9/9 [==============================] - 126s 14s/step - loss: 0.0056 - accuracy: 0.9983
Epoch 300/300
9/9 [==============================] - 124s 14s/step - loss: 0.0074 - accuracy: 0.9965


/home/ancao/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ancao/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Average Train Accuracy: 0.9991319444444444
Average Train AUC Score: 0.9999976423620389
Average Train MCC Score: 0.9986245809478488 


Average Train Classification Report:
              precision    recall  f1-score   support

    artifact    1.00000   1.00000   1.00000       128
    extrahls    1.00000   1.00000   1.00000        41
  extrastole    1.00000   1.00000   1.00000        70
      murmur    1.00000   0.99664   0.99832       298
      normal    0.99838   1.00000   0.99919       615

    accuracy                        0.99913      1152
   macro avg    0.99968   0.99933   0.99950      1152
weighted avg    0.99913   0.99913   0.99913      1152

Average Test Accuracy: 0.7370242214532872
Average Test AUC Score: 0.8624562259139747
Average Test MCC Score: 0.5738580379859861 


Average Test Classification Report:
              precision    recall  f1-score   support

    artifact    0.96875   0.96875   0.96875        32
    extrahls    0.66667   0.20000   0.30769        10
  extrasto

<Figure size 640x480 with 0 Axes>

In [9]:
from keras.models import load_model
model.save('model_CNN-LSTM.h5')

In [6]:
import os
import glob
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Audio
import librosa
import librosa.display

# Scikit learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import Input, add, Flatten, Dense, BatchNormalization, Dropout, LSTM, GRU
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalMaxPooling2D, Activation, LeakyReLU, ReLU, MaxPooling1D

# Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, BatchNormalization, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D

from tensorflow.keras.utils import to_categorical

# TensorFlow
from tensorflow.keras.utils import to_categorical

# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

dataset = []
for folder in ["Heartbeat_Sounds/set_a/**","Heartbeat_Sounds/set_b/**"]:
    for filename in glob.iglob(folder):
        if os.path.exists(filename):
            label = os.path.basename(filename).split("_")[0]
            duration = librosa.get_duration(filename=filename)
            # skip audio smaller than 3 secs
            if duration>=3:
                slice_size = 3
                iterations = int((duration-slice_size)/(slice_size-1))
                iterations += 1
#                 initial_offset = (duration % slice_size)/2
                initial_offset = (duration - ((iterations*(slice_size-1))+1))/2
                if label not in ["Aunlabelledtest", "Bunlabelledtest"]:
                    for i in range(iterations):
                        offset = initial_offset + i*(slice_size-1)
                        
                        dataset.append({
                                "filename": filename,
                                "label": label,
                                "offset": offset
                            })
                       
                        
dataset = pd.DataFrame(dataset)
dataset = shuffle(dataset, random_state=42)
dataset.info()

# Function to extract features
def extract_features(audio_path, offset):
    y, sr = librosa.load(audio_path, offset=offset, duration=3)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)
    return mfccs

# Extract features for all data points
x_data = []
for idx in tqdm(range(len(dataset))):
    x_data.append(extract_features(dataset.filename.iloc[idx], dataset.offset.iloc[idx]))

# Convert to numpy array
x_data = np.asarray(x_data)

# Encode Labels
encoder = LabelEncoder()
encoder.fit(dataset.label)
y_data = encoder.transform(dataset.label)

unique_labels = dataset['label'].unique()
num_classes = len(unique_labels)

# Compute class weights
class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_data), y=y_data)

y_data_one_hot = to_categorical(y_data, num_classes=num_classes)


# Initialize StratifiedKFold with train_size and test_size
kfold = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for each fold
train_accuracies = []
test_accuracies = []
train_auc_scores = []
test_auc_scores = []
train_mcc_scores = []
test_mcc_scores = []
train_conf_matrices = []
test_conf_matrices = []
train_class_reports = []
test_class_reports = []
cm_test_normalized = []
report_test_list = []
cm_test_normalized_list = []

# Open the common results file in 'w' mode to overwrite existing content
with open('results_heartSound/all_folds_lstm5_results200.txt', 'w') as common_file:
    # Split the indices instead of the dataset
    for fold, (train_index, test_index) in enumerate(kfold.split(x_data, y_data_one_hot), 1):
        x_train_fold, x_test_fold = x_data[train_index], x_data[test_index]
        y_train_fold, y_test_fold = y_data_one_hot[train_index], y_data_one_hot[test_index]

        model = Sequential()

        model.add(Conv1D(2048, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(40, 130)))
        model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
        model.add(BatchNormalization())

        model.add(Conv1D(1024, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
        model.add(BatchNormalization())  # This line was corrected from lstm_model to model

        model.add(Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
        model.add(BatchNormalization())

        model.add(LSTM(256, return_sequences=True))
        model.add(LSTM(128))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(num_classes, activation='softmax'))


        model.summary()

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        your_epochs = 100  # You can choose an appropriate number of epochs
        your_batch_size = 64  # You can choose an appropriate batch size

        # Train and evaluate the model for the current fold
        model.fit(x_train_fold, y_train_fold, epochs=your_epochs, batch_size=your_batch_size)
        y_pred_train = model.predict(x_train_fold)
        y_pred_test = model.predict(x_test_fold)
        
        y_pred_test_labels = np.argmax(y_pred_test, axis=1)
        report_test = classification_report(np.argmax(y_test_fold, axis=1), y_pred_test_labels, output_dict=True)

        # Calculate metrics for train set
        train_accuracy = accuracy_score(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_auc = roc_auc_score(y_train_fold, y_pred_train, multi_class='ovr')
        train_mcc = matthews_corrcoef(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_cm = confusion_matrix(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1))
        train_class_report = classification_report(np.argmax(y_train_fold, axis=1), np.argmax(y_pred_train, axis=1), target_names=encoder.classes_)

        # Calculate metrics for test set
        test_accuracy = accuracy_score(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_auc = roc_auc_score(y_test_fold, y_pred_test, multi_class='ovr')
        test_mcc = matthews_corrcoef(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))
        test_class_report = classification_report(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1), target_names=encoder.classes_)

        # Inside the loop where you append accuracies to lists
        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)

        # Calculate std for train and test accuracies
        train_accuracy_std = np.std(train_accuracies)
        test_accuracy_std = np.std(test_accuracies)


        # Save results to the common text file for both train and test sets
        common_file.write(f'Fold {fold} Results:\n')
        common_file.write(f'Train Accuracy: {train_accuracy} (std: {train_accuracy_std})\n')
        common_file.write(f'Train AUC Score: {train_auc}\n')
        common_file.write(f'Train MCC Score: {train_mcc}\n\n')
        common_file.write('Train Confusion Matrix:\n')
        common_file.write(str(train_cm))
        common_file.write('\n\nTrain Classification Report:\n')
        common_file.write(train_class_report)

        common_file.write(f'\n\nTest Accuracy: {test_accuracy} (std: {test_accuracy_std})\n')
        common_file.write(f'Test AUC Score: {test_auc}\n')
        common_file.write(f'Test MCC Score: {test_mcc}\n\n')
        common_file.write('Test Confusion Matrix:\n')
        common_file.write(str(test_cm))
        common_file.write('\n\nTest Classification Report:\n')
        common_file.write(test_class_report)
        
        # Print a separator between folds
        common_file.write("\n" + "="*40 + "\n")
        
        # Calculate average metrics for train set
        avg_train_accuracy = np.mean(train_accuracies)
        avg_train_auc = np.mean(train_auc_scores)
        avg_train_mcc = np.mean(train_mcc_scores)
        avg_train_conf_matrix = np.mean(train_conf_matrices, axis=0)  # Average confusion matrix

        # Convert continuous probabilities to class labels using argmax
        y_pred_train_class = np.argmax(y_pred_train, axis=1)

        # Calculate average metrics for test set
        avg_test_accuracy = np.mean(test_accuracies)
        avg_test_auc = np.mean(test_auc_scores)
        avg_test_mcc = np.mean(test_mcc_scores)
        avg_test_conf_matrix = np.mean(test_conf_matrices, axis=0)  # Average confusion matrix

        # Convert continuous probabilities to class labels using argmax
        y_pred_test_class = np.argmax(y_pred_test, axis=1)
        
        # Calculate average results for train set
        avg_train_class_report = classification_report(np.argmax(y_train_fold, axis=1), y_pred_train_class, target_names=encoder.classes_)

        # Calculate average results for test set
        avg_test_class_report = classification_report(np.argmax(y_test_fold, axis=1), y_pred_test_class, target_names=encoder.classes_)


        # Inside the loop where you plot confusion matrices
        plt.figure()
        train_cm = confusion_matrix(np.argmax(y_train_fold, axis=1), y_pred_train.argmax(axis=1))

        # Round each value in the confusion matrix to 5 decimal places
        rounded_train_cm = np.round(train_cm / np.sum(train_cm, axis=1)[:, np.newaxis], 5)

        sns.heatmap(rounded_train_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
        plt.title(f'Train Confusion Matrix - Fold {fold}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'LSTM200/train_confusion_matrix_fold_{fold}.png')
        plt.close()

        # Inside the loop where you plot confusion matrices
        plt.figure()
        test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), y_pred_test.argmax(axis=1))

        # Round each value in the confusion matrix to 5 decimal places
        rounded_test_cm = np.round(test_cm / np.sum(test_cm, axis=1)[:, np.newaxis], 5)

        sns.heatmap(rounded_test_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
        plt.title(f'Test Confusion Matrix - Fold {fold}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'LSTM200/test_confusion_matrix_fold_{fold}.png')
        plt.close()

        # Inside the loop where you append results to lists
        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)
        train_auc_scores.append(train_auc)
        test_auc_scores.append(test_auc)
        train_mcc_scores.append(train_mcc)
        test_mcc_scores.append(test_mcc)
        train_conf_matrices.append(train_cm)
        test_conf_matrices.append(test_cm)
        train_class_reports.append(train_class_report)
        test_class_reports.append(test_class_report)
        
        # Print a separator between folds
        common_file.write("\n" + "="*40 + "\n")
        
        # Append classification report and confusion matrix to lists
        report_test_list.append(report_test)
        cm_test_normalized_list.append(cm_test_normalized)
        

    # Calculate average metrics for train set
    avg_train_accuracy = np.mean(train_accuracies)
    avg_train_auc = np.mean(train_auc_scores)
    avg_train_mcc = np.mean(train_mcc_scores)

    # Reshape y_train_fold to (num_samples, num_classes)
    y_train_fold_reshaped = to_categorical(y_train_fold, num_classes=len(encoder.classes_))

    # Convert predictions to one-hot encoding
    y_pred_train_one_hot = to_categorical(np.argmax(y_pred_train, axis=1), num_classes=len(encoder.classes_))

    # Calculate average confusion matrix for train set
    avg_train_conf_matrix = np.mean(train_conf_matrices, axis=0).astype(int)


    # Calculate average classification report for train set
    avg_train_class_report = classification_report(
        np.argmax(y_train_fold, axis=1), 
        np.argmax(y_pred_train_one_hot, axis=1), 
        target_names=encoder.classes_, 
        digits=5
    )
    

    # Calculate average metrics for test set
    avg_test_accuracy = np.mean(test_accuracies)
    avg_test_auc = np.mean(test_auc_scores)
    avg_test_mcc = np.mean(test_mcc_scores)

    # Reshape y_test_fold to (num_samples, num_classes)
    y_test_fold_reshaped = to_categorical(y_test_fold, num_classes=len(encoder.classes_))

    # Convert predictions to one-hot encoding
    y_pred_test_one_hot = to_categorical(np.argmax(y_pred_test, axis=1), num_classes=len(encoder.classes_))

    # Calculate average confusion matrix for test set
    avg_test_conf_matrix = np.mean(test_conf_matrices, axis=0).astype(int)

    # Calculate average classification report for test set
    avg_test_class_report = classification_report(
        np.argmax(y_test_fold, axis=1), 
        np.argmax(y_pred_test_one_hot, axis=1), 
        target_names=encoder.classes_, 
        digits=5
    )

    # Inside the loop where you plot the average confusion matrix for the test set
    avg_cm_test_normalized = np.mean(cm_test_normalized_list, axis=0)
    plt.figure()

    avg_test_cm = confusion_matrix(np.argmax(y_test_fold, axis=1), np.argmax(y_pred_test, axis=1))

    # Normalize the confusion matrix by dividing each value by the sum of its row
    normalized_avg_test_cm = avg_test_cm / avg_test_cm.sum(axis=1)[:, np.newaxis]

    # Plot confusion matrix for the average test set
    plt.figure()
    sns.heatmap(normalized_avg_test_cm, annot=True, fmt='.5f', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
    plt.title(f'Average Test Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('LSTM200/average_test_confusion_matrix.png')
    plt.close()



    # Save average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy:.5f} (std: {np.std(test_accuracies):.5f})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc:.5f}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc:.5f}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_cm_test_normalized.round(5).astype(int)))  # Display confusion matrix with 5 decimal places
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))


    # Print and write the average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy:.5f} (std: {np.std(test_accuracies):.5f})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc:.5f}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc:.5f}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_cm_test_normalized.round(5).astype(int)))  # Display confusion matrix with 5 decimal places
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))


    # Print and write the average results to the common text file
    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Train Accuracy: {avg_train_accuracy} (std: {np.std(train_accuracies)})\n')
    common_file.write(f'Average Train AUC Score: {avg_train_auc}\n')
    common_file.write(f'Average Train MCC Score: {avg_train_mcc}\n\n')
    common_file.write('Average Train Confusion Matrix:\n')
    common_file.write(str(avg_train_conf_matrix.astype(int)))
    common_file.write("\n\nAverage Train Classification Report:\n")
    common_file.write(str(avg_train_class_report))

    common_file.write("\n" + "="*40 + "\n")
    common_file.write(f'Average Test Accuracy: {avg_test_accuracy} (std: {np.std(test_accuracies)})\n')
    common_file.write(f'Average Test AUC Score: {avg_test_auc}\n')
    common_file.write(f'Average Test MCC Score: {avg_test_mcc}\n\n')
    common_file.write('Average Test Confusion Matrix:\n')
    common_file.write(str(avg_test_conf_matrix.astype(int)))
    common_file.write("\n\nAverage Test Classification Report:\n")
    common_file.write(str(avg_test_class_report))
        
    # Print average results for train set
    print(f'Average Train Accuracy: {avg_train_accuracy}')
    print(f'Average Train AUC Score: {avg_train_auc}')
    print(f'Average Train MCC Score: {avg_train_mcc} \n')
    print("\nAverage Train Classification Report:")
    print(avg_train_class_report)

    # Print average results for test set
    print(f'Average Test Accuracy: {avg_test_accuracy}')
    print(f'Average Test AUC Score: {avg_test_auc}')
    print(f'Average Test MCC Score: {avg_test_mcc} \n')
    print("\nAverage Test Classification Report:")
    print(avg_test_class_report)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1441 entries, 168 to 1126
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  1441 non-null   object 
 1   label     1441 non-null   object 
 2   offset    1441 non-null   float64
dtypes: float64(1), object(2)
memory usage: 45.0+ KB


100%|██████████| 1441/1441 [00:30<00:00, 46.74it/s]


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 40, 2048)          1333248   
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 20, 2048)          0         
 ng1D)                                                           
                                                                 
 batch_normalization_12 (Ba  (None, 20, 2048)          8192      
 tchNormalization)                                               
                                                                 
 conv1d_13 (Conv1D)          (None, 20, 1024)          10486784  
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 10, 1024)          0         
 ng1D)                                                           
                                                      

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 40, 2048)          1333248   
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 20, 2048)          0         
 ng1D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 20, 2048)          8192      
 tchNormalization)                                               
                                                                 
 conv1d_16 (Conv1D)          (None, 20, 1024)          10486784  
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 10, 1024)          0         
 ng1D)                                                           
                                                      

...

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_30 (Conv1D)          (None, 40, 2048)          1333248   
                                                                 
 max_pooling1d_30 (MaxPooli  (None, 20, 2048)          0         
 ng1D)                                                           
                                                                 
 batch_normalization_30 (Ba  (None, 20, 2048)          8192      
 tchNormalization)                                               
                                                                 
 conv1d_31 (Conv1D)          (None, 20, 1024)          10486784  
                                                                 
 max_pooling1d_31 (MaxPooli  (None, 10, 1024)          0         
 ng1D)                                                           
                                                     

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Average Train Accuracy: 0.7848090277777777
Average Train AUC Score: 0.9174008808574392
Average Train MCC Score: 0.6611743333408496 


Average Train Classification Report:
              precision    recall  f1-score   support

    artifact    0.95522   1.00000   0.97710       128
    extrahls    0.68085   0.78049   0.72727        41
  extrastole    0.00000   0.00000   0.00000        70
      murmur    0.75556   0.11409   0.19825       298
      normal    0.63931   0.96260   0.76833       615

    accuracy                        0.68229      1152
   macro avg    0.60619   0.57144   0.53419      1152
weighted avg    0.66711   0.68229   0.59591      1152

Average Test Accuracy: 0.6923875432525952
Average Test AUC Score: 0.8181325816138647
Average Test MCC Score: 0.5003284958323775 


Average Test Classification Report:
              precision    recall  f1-score   support

    artifact    0.94118   1.00000   0.96970        32
    extrahls    0.50000   0.70000   0.58333        10
  extrasto

<Figure size 640x480 with 0 Axes>